In [15]:
import requests
import os
from collections import Counter, defaultdict
from itertools import chain, combinations

In [16]:
def download_document(file_name, document_url):

   if os.path.exists(file_name):

       pass

   else:

       response = requests.get(document_url)

       if response.status_code == 200:

           with open(file_name, 'wb') as f:

               f.write(response.content)

       else:

           print(f'Failed to download the document. Status code: {response.status_code}')

In [17]:
file_name = 'orders.txt'

document_url = 'https://drive.google.com/uc?id=1IOPTVq2ooQfZRkF3rAjGkTjRtbotG7FF'

download_document(file_name, document_url)

In [18]:
def read_orders(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        orders = file.read().split('\n\n')
        orders = [order.split('@@@') for order in orders]
    return orders

In [19]:
def count_products(orders):
    product_count = Counter(chain.from_iterable(orders))
    return product_count

In [20]:
def calculate_support(orders, product1, product2):
    support_count = sum(1 for order in orders if product1 in order and product2 in order)
    support = support_count / len(orders)
    return support

In [21]:
def generate_association_rules(orders, min_confidence, min_support):
    product_count = count_products(orders)
    association_rules = []

    for order in orders:
        for product1, product2 in combinations(order, 2):
            support = calculate_support(orders, product1, product2)
            if support >= min_support:
                confidence1 = support / product_count[product1]
                confidence2 = support / product_count[product2]

                if confidence1 >= min_confidence:
                    association_rules.append((product1, product2, support, confidence1))
                if confidence2 >= min_confidence:
                    association_rules.append((product2, product1, support, confidence2))

    for rule in association_rules:
        print(f"Association Rule: {rule[0]} -> {rule[1]}, Support: {rule[2]}, Confidence: {rule[3]}")

In [23]:
orders = read_orders(file_name)
min_confidence = 0.5
min_support = 0.2
generate_association_rules(orders, min_confidence, min_support)

KeyboardInterrupt: 